In [1]:
require 'rnn'
require 'itorch.Plot'

In [2]:
-- TODO something less hardcoded
filenames = {}
for i=2,9 do
    for _, typ in ipairs({'full', 'lstm', 'rnn', 'fulltwo'}) do
        table.insert(filenames, string.format('models_%d_%s/rubiks_best', i, typ))
    end
end

best_results = {}
for _, name in ipairs(filenames) do
    best_results[name] = torch.load(name, 'ascii')
    print(name, 'loaded')
end

models_2_full/rubiks_best	loaded	


models_2_lstm/rubiks_best	loaded	


models_2_rnn/rubiks_best	

loaded	


models_2_fulltwo/rubiks_best	loaded	


...

models_8_fulltwo/rubiks_best	loaded	


models_9_full/rubiks_best	loaded	


models_9_lstm/rubiks_best	loaded	


models_9_rnn/rubiks_best	loaded	


models_9_fulltwo/rubiks_best	loaded	


In [4]:
-- Training accuracy plot
plot = itorch.Plot()
x = torch.range(2, 9)
full_y = torch.Tensor(8)
full2_y = torch.Tensor(8)
rnn_y = torch.Tensor(8)
lstm_y = torch.Tensor(8)
for i=2,9 do
    name = string.format('models_%d_full/rubiks_best', i)
    full_y[i-1] = best_results[name].train_acc
    name = string.format('models_%d_fulltwo/rubiks_best', i)
    full2_y[i-1] = best_results[name].train_acc
    name = string.format('models_%d_rnn/rubiks_best', i)
    rnn_y[i-1] = best_results[name].train_acc
    name = string.format('models_%d_lstm/rubiks_best', i)
    lstm_y[i-1] = best_results[name].train_acc
end

plot:xaxis('Episode Length'):yaxis('Training Accuracy')
plot:title('Training accuracy (50000 episodes)')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, full2_y, 'pink', 'Fully Connected 2')
plot:line(x, rnn_y, 'blue', 'RNN')
plot:line(x, lstm_y, 'green', 'LSTM')
plot:legend(true)
plot:draw()

In [5]:
-- Test accuracy plot
plot = itorch.Plot()
x = torch.range(2, 9)
full_y = torch.Tensor(8)
rnn_y = torch.Tensor(8)
lstm_y = torch.Tensor(8)
for i=2,9 do
    name = string.format('models_%d_full/rubiks_best', i)
    full_y[i-1] = best_results[name].test_acc
    name = string.format('models_%d_fulltwo/rubiks_best', i)
    full2_y[i-1] = best_results[name].test_acc
    name = string.format('models_%d_rnn/rubiks_best', i)
    rnn_y[i-1] = best_results[name].test_acc
    name = string.format('models_%d_lstm/rubiks_best', i)
    lstm_y[i-1] = best_results[name].test_acc
end

plot:xaxis('Episode Length'):yaxis('Test Accuracy')
plot:title('Test accuracy (50000 episodes)')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, full2_y, 'pink', 'Fully Connected 2')
plot:line(x, rnn_y, 'blue', 'RNN')
plot:line(x, lstm_y, 'green', 'LSTM')
plot:legend(true)
plot:draw()